In [3]:
!pip install qiskit qiskit_aer qiskit_ibm_runtime matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.4/377.4 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.7 MB/s eta 0:00:00


In [8]:
# --- IMPORTS ---
# General Qiskit and plotting
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import warnings

# Imports for Task 3
try:
    from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session
    IBM_RUNTIME_INSTALLED = True
except ImportError:
    print("qiskit_ibm_runtime not found. Task 3 will be skipped.")
    IBM_RUNTIME_INSTALLED = False

# Imports for Task 4
try:
    from qiskit_aer.noise import NoiseModel, depolarizing_error
    NOISE_MODEL_INSTALLED = True
except ImportError:
    print("qiskit_aer.noise not found. Task 4 will be skipped.")
    NOISE_MODEL_INSTALLED = False


# --- BASE CODE AND FIX ---
# This is your original code from the notebook, with the bug fixed.

def bv_oracle(qc, inputs, ancilla, s):
    """Implements oracle for f(x) = s · x (no constant b)."""
    # This function is unchanged from your original code
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(inputs[i], ancilla)

def bernstein_vazirani_circuit(s):
    """Builds the circuit for f(x) = s · x."""
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n

    qc.x(ancilla)
    qc.h(qreg)

    # --- THE FIX ---
    # We reverse 's' to fix the Qiskit endianness (qubit ordering) bug.
    bv_oracle(qc, inputs, ancilla, s[::-1])
    # --- END FIX ---

    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def run_bv(qc, shots=1024):
    """Runs the circuit on the ideal AerSimulator."""
    sim = AerSimulator()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Counts:', counts)
    fig = plot_histogram(counts)
    plt.show() # This will display the plot
    most = max(counts, key=counts.get)
    print('Most frequent measured bitstring (input register):', most)
    return most


# --- Task 1: Change the Secret String ---
def run_task_1():
    print("--- Running Task 1: Change the Secret String ---")
    s = '11001' # Using a different string as per task
    print(f'Secret string s = {s}')
    qc = bernstein_vazirani_circuit(s)
    print("Circuit diagram for Task 1:")
    print(qc.draw(fold=-1))
    measured = run_bv(qc)
    if measured == s:
        print(f'✅ Successfully recovered secret string s: {measured}')
    else:
        print(f'⚠️ Measured string ({measured}) differs from s ({s}).')
    print("--------------------------------------------------\n")

# --- Task 2: Modify Oracle for Constant b ---
def bv_oracle_with_b(qc, inputs, ancilla, s_reversed, b):
    """Implements oracle for f(x) = s · x ⊕ b."""
    for i, bit in enumerate(s_reversed):
        if bit == '1':
            qc.cx(inputs[i], ancilla)
    if b == '1':
        qc.z(ancilla) # Apply phase kickback for constant b=1

def bernstein_vazirani_circuit_with_b(s, b='0'):
    """Builds the circuit for f(x) = s · x ⊕ b."""
    n = len(s)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)
    inputs = list(range(n))
    ancilla = n
    qc.x(ancilla)
    qc.h(qreg)
    bv_oracle_with_b(qc, inputs, ancilla, s[::-1], b)
    for q in inputs:
        qc.h(q)
    qc.measure(inputs, creg)
    return qc

def run_task_2():
    print("--- Running Task 2: Modify Oracle for Constant b ---")
    s = '1011'
    b = '1' # Set the constant b
    print(f'Secret string s = {s}, Constant b = {b}')
    qc = bernstein_vazirani_circuit_with_b(s, b)
    print("Circuit diagram for Task 2 (with b=1):")
    print(qc.draw(fold=-1))
    measured = run_bv(qc)
    if measured == s:
        print(f'✅ Successfully recovered s: {measured} (b does not affect input measurement)')
    else:
        print(f'⚠️ Measured string ({measured}) differs from s ({s}).')
    print("--------------------------------------------------\n")

# --- Task 3: Run on a Real IBM Backend ---
def run_on_ibm(qc, token):
    """Runs the circuit on a real IBM backend using Sampler."""
    try:
        service = QiskitRuntimeService(channel="ibm_quantum", token=token)
        backend = service.least_busy(min_num_qubits=qc.num_qubits)
        print(f"Using real backend: {backend.name}")
        with Session(service=service, backend=backend) as session:
            sampler = Sampler(session=session)
            qc_no_measure = qc.remove_final_measurements(inplace=False)
            print("Running job on real hardware...")
            job = sampler.run([qc_no_measure])
            print(f"Job ID: {job.job_id()}")
            result = job.result()
            data = result[0].data
            counts = data.c.get_counts()
            print('Counts:', counts)
            plot_histogram(counts)
            plt.show()
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please ensure your API token is correct.")

def run_task_3():
    print("--- Running Task 3: Run on a Real IBM Backend ---")
    s = '101' # Use a small string for real hardware
    qc = bernstein_vazirani_circuit(s)

    # --- PASTE YOUR IBM QUANTUM API TOKEN HERE ---
    my_token = 'PASTE_YOUR_API_TOKEN_HERE'

    # --- ADDED DIAGRAM PRINT FOR TASK 3 ---
    print("Circuit diagram for Task 3:")
    print(qc.draw(fold=-1))

    if my_token != 'PASTE_YOUR_API_TOKEN_HERE':
        if IBM_RUNTIME_INSTALLED:
            run_on_ibm(qc, my_token)
        else:
            print("Skipping Task 3, qiskit_ibm_runtime not installed.")
    else:
        print("Task 3 skipped: Please paste your IBM Quantum API token to run.")
    print("--------------------------------------------------\n")

# --- Task 4: Add Noise ---
def create_noise_model():
    """Creates a simple depolarizing noise model."""
    p_gate1 = 0.005 # 0.5% chance of a single-qubit error
    p_gate2 = 0.05  # 5% chance of a two-qubit error
    error_1 = depolarizing_error(p_gate1, 1)
    error_2 = depolarizing_error(p_gate2, 2)
    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_1, ['h'])
    noise_model.add_all_qubit_quantum_error(error_2, ['cx'])
    return noise_model

def run_bv_noisy(qc, shots=1024, noise_model=None):
    """Runs the circuit on a noisy AerSimulator."""
    sim = AerSimulator(noise_model=noise_model)
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    print('Noisy Counts:', counts)
    fig = plot_histogram(counts)
    plt.show() # This will display the plot for the noisy run
    most = max(counts, key=counts.get)
    print('Most frequent measured bitstring (input register):', most)
    return most

def run_task_4():
    print("--- Running Task 4: Add Noise ---")
    s = '1011'
    print(f'Secret string s = {s}')

    qc = bernstein_vazirani_circuit(s)

    # --- ADDED DIAGRAM PRINT FOR TASK 4 ---
    print("Circuit diagram for Task 4:")
    print(qc.draw(fold=-1))

    if NOISE_MODEL_INSTALLED:
        noise = create_noise_model()
        print("Running with noise...")
        measured = run_bv_noisy(qc, noise_model=noise)
        if measured == s:
            print(f'✅ Successfully recovered secret string s: {measured} (despite noise)')
        else:
            print(f'⚠️ Measured string ({measured}) differs from s ({s}) due to noise.')
    else:
        print("Skipping Task 4, qiskit_aer.noise not imported.")
    print("--------------------------------------------------\n")

# --- Task 5: Create a Notebook (Explanation) ---
def run_task_5():
    print("--- Task 5: Explanation ---")
    print("This task is the explanation text. This is the 'output' for Task 5.")
    print("Here is the explanation text to add to a markdown cell:")
    # Using triple quotes to print a multi-line string
    print(
"""
The Goal: Find a hidden n-bit string s (e.g., '101') inside a "black box"
function f(x) = s · x (mod 2). The algorithm finds s in a single query.

The Circuit Steps:

1.  Initialization: We need n+1 qubits.
    * n qubits for the input register (initially |00...0>).
    * 1 ancilla (helper) qubit for the output (initially |0>).

2.  Ancilla Preparation: We put the ancilla into the state |->.
    * qc.x(ancilla): Flips |0> -> |1>.
    * qc.h(ancilla): Applies Hadamard, turning |1> -> (|0> - |1>)/sqrt(2),
      which is the |-> state.
    * Why? This state is how we achieve "phase kickback."

3.  Create Superposition: We apply a Hadamard (H) gate to all n input qubits.
    * qc.h(inputs): This puts the input register into an equal superposition
      of all 2^n possible bitstrings.

4.  The Oracle (One Query): This is the "black box" bv_oracle.
    * For each bit i where s[i] == '1', a CNOT gate is applied from
      input qubit i to the ancilla.
    * Phase Kickback: Because the ancilla is in the |-> state, any CNOT
      kicks back a negative phase (a Z gate) to its corresponding input qubit.
    * After the oracle, the state of the input qubits is encoded with 's'
      in its phases.

5.  Hadamard Transform: We apply H gates to all n input qubits again.
    * This second set of Hadamards converts the state from the "phase basis"
      back to the "computational basis."
    * This transformation magically turns the state into the single,
      simple state |s>.

6.  Measurement: We measure the n input qubits.
    * qc.measure(inputs, creg)
    * Because the input qubits are now in the state |s> (e.g., |101>),
      measuring them will always return the bitstring s (in an ideal,
      noiseless simulator).
"""
    )
    print("--------------------------------------------------\n")


# --- MAIN EXECUTION BLOCK ---
# This block will now run all 5 tasks in sequence.
if __name__ == '__main__':
    # Suppress unnecessary warnings
    warnings.filterwarnings("ignore", category=UserWarning)

    run_task_1()
    run_task_2()
    run_task_3()
    run_task_4()
    run_task_5()

    # Restore warnings
    warnings.filterwarnings("default", category=UserWarning)

--- Running Task 1: Change the Secret String ---
Secret string s = 11001
Circuit diagram for Task 1:
     ┌───┐          ┌───┐             ┌─┐           
q_0: ┤ H ├───────■──┤ H ├─────────────┤M├───────────
     ├───┤┌───┐  │  └┬─┬┘             └╥┘           
q_1: ┤ H ├┤ H ├──┼───┤M├───────────────╫────────────
     ├───┤├───┤  │   └╥┘ ┌─┐           ║            
q_2: ┤ H ├┤ H ├──┼────╫──┤M├───────────╫────────────
     ├───┤└───┘  │    ║  └╥┘     ┌───┐ ║      ┌─┐   
q_3: ┤ H ├───────┼────╫───╫───■──┤ H ├─╫──────┤M├───
     ├───┤       │    ║   ║   │  └───┘ ║ ┌───┐└╥┘┌─┐
q_4: ┤ H ├───────┼────╫───╫───┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║   ║ ┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_5: ┤ X ├┤ H ├┤ X ├──╫───╫─┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║   ║ └───┘└───┘ ║       ║  ║ 
c: 5/═════════════════╩═══╩════════════╩═══════╩══╩═
                      1   2            0       3  4 
Counts: {'11001': 1024}
Most frequent measured bitstring (input register): 11001
✅ Successfully recovere